In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

In [62]:
#leemos los datos
data = pd.read_csv("../dataset/data_evaluacion.csv")
data.head(5)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [63]:
#Asiganmos nombres a las columnas sin etiquetas
data.columns = [
    'edad',                # age
    'clase_laboral',       # workclass
    'fnlwgt',              # continuous (final weight)
    'educacion',           # education
    'anios_educacion_dedicados',       # continuous (education-num)
    'estado_civil',        # marital-status
    'ocupacion',           # occupation
    'relacion',            # relationship
    'raza',                # race
    'sexo',                # sex
    'ganancia_capital',    # continuous (capital-gain)
    'perdida_capital',     # continuous (capital-loss)
    'horas_por_semana',    # continuous (hours-per-week)
    'pais_origen',         # native-country
    'ingreso'              # income
]
data.head(5)

,edad,clase_laboral,fnlwgt,educacion,anios_educacion_dedicados,estado_civil,ocupacion,relacion,raza,sexo,ganancia_capital,perdida_capital,horas_por_semana,pais_origen,ingreso
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [64]:
#verificamos la cantidad de filas y columnas

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48841 entries, 0 to 48840
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   edad                       48841 non-null  int64 
 1   clase_laboral              48841 non-null  object
 2   fnlwgt                     48841 non-null  int64 
 3   educacion                  48841 non-null  object
 4   anios_educacion_dedicados  48841 non-null  int64 
 5   estado_civil               48841 non-null  object
 6   ocupacion                  48841 non-null  object
 7   relacion                   48841 non-null  object
 8   raza                       48841 non-null  object
 9   sexo                       48841 non-null  object
 10  ganancia_capital           48841 non-null  int64 
 11  perdida_capital            48841 non-null  int64 
 12  horas_por_semana           48841 non-null  int64 
 13  pais_origen                48841 non-null  object
 14  ingres

In [65]:
#verificamos datos nulos
data.isnull().sum()

edad                         0
clase_laboral                0
fnlwgt                       0
educacion                    0
anios_educacion_dedicados    0
estado_civil                 0
ocupacion                    0
relacion                     0
raza                         0
sexo                         0
ganancia_capital             0
perdida_capital              0
horas_por_semana             0
pais_origen                  0
ingreso                      0
dtype: int64

In [66]:
#Mínimos y máximos de categorías numéricas
data.describe().T

,count,mean,std,min,25%,50%,75%,max
edad,48841.0,38.643578,13.710650,17.0,28.0,37.0,48.0,90.0
fnlwgt,48841.0,189666.430786,105603.887256,12285.0,117555.0,178147.0,237646.0,1490400.0
anios_educacion_dedicados,48841.0,10.078029,2.570965,1.0,9.0,10.0,12.0,16.0
ganancia_capital,48841.0,1079.045208,7452.093700,0.0,0.0,0.0,0.0,99999.0
perdida_capital,48841.0,87.504105,403.008483,0.0,0.0,0.0,0.0,4356.0
horas_por_semana,48841.0,40.422391,12.391571,1.0,40.0,40.0,45.0,99.0


In [67]:
data.head(5)

,edad,clase_laboral,fnlwgt,educacion,anios_educacion_dedicados,estado_civil,ocupacion,relacion,raza,sexo,ganancia_capital,perdida_capital,horas_por_semana,pais_origen,ingreso
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [68]:
categorical = ['clase_laboral','educacion','estado_civil','ocupacion','relacion','raza','sexo','pais_origen']
label_encoder = LabelEncoder()
for col in categorical:
    label_encoder.fit(data[col])
    data[col] = label_encoder.transform(data[col])
    

data.head(5) 

,edad,clase_laboral,fnlwgt,educacion,anios_educacion_dedicados,estado_civil,ocupacion,relacion,raza,sexo,ganancia_capital,perdida_capital,horas_por_semana,pais_origen,ingreso
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,<=50K
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,<=50K
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,<=50K
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,<=50K
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,<=50K


In [69]:
#Caracteristicas y variable objetivo
x = data.drop(columns=['ingreso'], axis=1)
y = data['ingreso']


In [70]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [71]:
scaler = StandardScaler()

x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns = x.columns)
x_test_scaled = pd.DataFrame(scaler.transform(x_test), columns = x.columns)

In [72]:
k_range = range(1, 31)
k_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train_scaled, y_train, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())

KeyboardInterrupt: 

In [ ]:
# Graficar los resultados
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.title('Accuracy vs. K Value')
plt.show()

In [ ]:
# Imprimir el mejor valor de K
best_k = k_range[k_scores.index(max(k_scores))]
print(f"El mejor valor de k es: {best_k}")


In [ ]:
# Evalaucion del modelo
# Predecir en el conjunto de prueba

knn = KNeighborsClassifier(n_neighbors=best_k)  # Puedes ajustar el número de vecinos
knn.fit(x_train_scaled, y_train)

# Predecir en el conjunto de prueba
y_pred = knn.predict(x_test_scaled)
print('Train Accuracy : %.5f' % knn.score(x_train_scaled, y_train))
print('Test Accuracy : %.5f' % knn.score(x_test_scaled, y_test))
print("Precisión del modelo:", accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))

# si la diferencia entre train y test del modelo es mas de 10 es porque tiene underfiting (le faltan datos)
# si la diferencia entre test y precisión del modelo es mas de 10 es porque tiene overfiting (el modelo esta memorizando)

Train Accuracy : 0.87885
Test Accuracy : 0.82454
Precisión del modelo: 0.7614140449054801
              precision    recall  f1-score   support

       <=50K       0.76      1.00      0.86     11129
        >50K       1.00      0.01      0.02      3524

    accuracy                           0.76     14653
   macro avg       0.88      0.50      0.44     14653
weighted avg       0.82      0.76      0.66     14653



In [ ]:
# Mostrar la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_test, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Conjunto de Prueba')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

Aditimulye. (2021, July 4). Adult Income Dataset | From scratch. https://www.kaggle.com/code/aditimulye/adult-income-dataset-from-scratch